In [ ]:
import fxcmpy
import time
import datetime as dt
from pyti.simple_moving_average import simple_moving_average as sma
from pyti.exponential_moving_average import exponential_moving_average as ema

### STRATEGY DESCRIPTION ####
# This strategy buys when the Fast SMA crosses over the Slow SMA and sell when the Fast SMA crosses
# under the Slow SMA. Opposing signals will close out opposing positions if close_on_opposing_signal == True.
# If close_on_opposing_signal == False, the strategy can open hedged positions (long & short at the same time)
# Parameters allow traders to change token, symbol, timeframe, Fast SMA & Slow SMA Periods, 
# Opposing signal closing logic, trade size/stop/limit.
# This is a close-of-bar strategy, meaning it only signals trades at the close of a bar.
# For more strategy examples, please visit github.com/fxcm/RestAPI
#############################

###### USER PARAMETERS ######
token = '817d90e62163c99e1dcf4f03b7cfadd52222ced6'
symbol = 'EUR/USD'
timeframe = "m5"	        # (m1,m5,m15,m30,H1,H2,H3,H4,H6,H8,D1,W1,M1)
fast_sma_periods = 10
slow_sma_periods = 20
close_on_opposing_signal = True
amount = 1
stop = -10
limit = 30
#############################

# Global Variables
pricedata = None
numberofcandles = 300

# Connect to FXCM API
con = fxcmpy.fxcmpy(access_token="817d90e62163c99e1dcf4f03b7cfadd52222ced6", log_level="error")
# con = fxcmpy.fxcmpy(config_file='fxcm.cfg')
	
# This function runs once at the beginning of the strategy to run initial one-time processes/computations
def Prepare():
	global pricedata
	
	print("Requesting Initial Price Data...")
	pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	print(pricedata)
	print("Initial Price Data Received...")

# Get latest close bar prices and run Update() function every close of bar/candle
def StrategyHeartBeat():
	while True:
		currenttime = dt.datetime.now()
		if timeframe == "m1" and currenttime.second == 0 and GetLatestPriceData():
			Update()
		elif timeframe == "m5" and currenttime.second == 0 and currenttime.minute % 5 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(240)
		elif timeframe == "m15" and currenttime.second == 0 and currenttime.minute % 15 == 0 and GetLatestPriceData(): 
			Update()
			time.sleep(840)
		elif timeframe == "m30" and currenttime.second == 0 and currenttime.minute % 30 == 0 and GetLatestPriceData():
			Update()
			time.sleep(1740)
		elif currenttime.second == 0 and currenttime.minute == 0 and GetLatestPriceData():
			Update()
			time.sleep(3540)
		time.sleep(1)

# Returns True when pricedata is properly updated			
def GetLatestPriceData():
	global pricedata
	
	# Normal operation will update pricedata on first attempt
	new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata= new_pricedata
		return True
		
	counter = 0
	# If data is not available on first attempt, try up to 3 times to update pricedata
	while new_pricedata.index.values[len(new_pricedata.index.values)-1] == pricedata.index.values[len(pricedata.index.values)-1] and counter < 3:
		print("No updated prices found, trying again in 10 seconds...")
		counter+=1
		time.sleep(10)
		new_pricedata = con.get_candles(symbol, period=timeframe, number=numberofcandles)
	if new_pricedata.index.values[len(new_pricedata.index.values)-1] != pricedata.index.values[len(pricedata.index.values)-1]:
		pricedata = new_pricedata
		return True
	else:
		return False
		
# This function is run every time a candle closes
def Update():
	print(str(dt.datetime.now()) + "	 " + timeframe + " Bar Closed - Running Update Function...")
    
	colfastL = False
	colfastS = False
    
	colslowL = False
	colslowS = False
    
	colFinal = ""
	colFinal2 = ""
    
    
    #Fast EMA ARRAYS
	ema1array = ema(pricedata['bidclose'], 3)
	ema2array = ema(pricedata['bidclose'], 6)
	ema3array = ema(pricedata['bidclose'], 9)
	ema4array = ema(pricedata['bidclose'], 12)
	ema5array = ema(pricedata['bidclose'], 15)
	ema6array = ema(pricedata['bidclose'], 18)
	ema7array = ema(pricedata['bidclose'], 21)
    
    #Slow EMA ARRAYS
	ema8array = ema(pricedata['bidclose'], 24)
	ema9array = ema(pricedata['bidclose'], 27)
	ema10array = ema(pricedata['bidclose'], 30)
	ema11array = ema(pricedata['bidclose'], 33)
	ema12array = ema(pricedata['bidclose'], 36)
	ema13array = ema(pricedata['bidclose'], 39)
	ema14array = ema(pricedata['bidclose'], 42)
	ema15array = ema(pricedata['bidclose'], 45)
	ema16array = ema(pricedata['bidclose'], 48)
	ema17array = ema(pricedata['bidclose'], 51)
	ema18array = ema(pricedata['bidclose'], 54)
	ema19array = ema(pricedata['bidclose'], 57)
	ema20array = ema(pricedata['bidclose'], 60)
	ema21array = ema(pricedata['bidclose'], 63)
	ema22array = ema(pricedata['bidclose'], 66)
    
    #EMA 200 ARRAY
	ema23array = ema(pricedata['bidclose'], 200)
    
    
    #Fast EMA
	ema1 = ema1array[len(ema1array)-1]
	ema2 = ema2array[len(ema2array)-1]
	ema3 = ema3array[len(ema3array)-1]
	ema4 = ema4array[len(ema4array)-1]
	ema5 = ema5array[len(ema5array)-1]
	ema6 = ema6array[len(ema6array)-1]
	ema7 = ema7array[len(ema7array)-1]
    
    
    #Slow EMA
	ema8 = ema8array[len(ema8array)-1]
	ema9 = ema9array[len(ema9array)-1]
	ema10 = ema10array[len(ema10array)-1]
	ema11 = ema11array[len(ema11array)-1]
	ema12 = ema12array[len(ema12array)-1]
	ema13 = ema13array[len(ema13array)-1]
	ema14 = ema14array[len(ema14array)-1]
	ema15 = ema15array[len(ema15array)-1]
	ema16 = ema16array[len(ema16array)-1]
	ema17 = ema17array[len(ema17array)-1]
	ema18 = ema18array[len(ema18array)-1]
	ema19 = ema19array[len(ema19array)-1]
	ema20 = ema20array[len(ema20array)-1]
	ema21 = ema21array[len(ema21array)-1]
	ema22 = ema22array[len(ema22array)-1]
    
    #EMA 200
	ema23 = ema23array[len(ema23array)-1]
    
    #Fast EMA Color Rules
	if ema1 > ema2 and ema2 > ema3 and ema3 > ema4 and ema4 > ema5 and ema5 > ema6 and ema6 > ema7:
		colfastL = True
    
	if ema1 < ema2 and ema2 < ema3 and ema3 < ema4 and ema4 < ema5 and ema5 < ema6 and ema6 < ema7:
		colfastS = True
    
    #Slow EMA Color Rules
	if ema8 > ema9 and ema9 > ema10 and ema10 > ema11 and ema11 > ema12 and ema12 > ema13 and ema13 > ema14 and ema14 > ema15 and ema15 > ema16 and ema16 > ema17 and ema17 > ema18 and ema18 > ema19 and ema19 > ema20 and ema20 > ema21 and ema21 > ema22:
		colslowL = True
    
	if ema8 < ema9 and ema9 < ema10 and ema10 < ema11 and ema11 < ema12 and ema12 < ema13 and ema13 < ema14 and ema14 < ema15 and ema15 < ema16 and ema16 < ema17 and ema17 < ema18 and ema18 < ema19 and ema19 < ema20 and ema20 < ema21 and ema21 < ema22:
		colslowS = True
    
    #Fast EMA Final Color Rules
	if colfastL and colslowL:
		colFinal = "aqua"
	elif colfastS and colslowS:
		colFinal = "orange"
	else:
		colFinal = "gray"
        
    #Slow EMA Final Color Rules
	if colslowL:
		colFinal2 = "lime"
	elif colslowS:
		colFinal2 = "red"
	else:
		colFinal2 = "gray"
        
    #TRADING LOGIC
    #Buy If colFinal is aqua and colFinal2 is lime
	if colFinal == "aqua" and colFinal2 == "lime":
		print("	  BUY SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("S") > 0:
			print("	  Closing Sell Trade(s)...")
			exit("S")
		print("	  Opening Buy Trade...")
		enter("B")
        
    #Sell If colFinal is orange and colFinal2 is red
	if colFinal == "orange" and colFinal2 == "red":
		print("	  SELL SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("B") > 0:
			print("	  Closing Buy Trade(s)...")
			exit("B")
		print("	  Opening Sell Trade...")
		enter("S")
        
    #EXIT STRTEGY
	if colFinal == "grey":
		print("	  EXIT SELL SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("S") > 0:
			print("	  Closing Sell Trade(s)...")
			exit("S")
		print("	  EXIT BUY SIGNAL!")
		if close_on_opposing_signal and countOpenTrades("B") > 0:
			print("	  Closing Buy Trade(s)...")
			exit("B")
            
    
        
	
	# Print Price/Indicators
	print("Close Price: " + str(pricedata['bidclose'][len(pricedata)-1]))
	print("ColFinal: " + colFinal)
	print("ColFinal2: " + colFinal2)
	
		
	print(str(dt.datetime.now()) + "	 " + timeframe + " Update Function Completed.\n")

# This function places a market order in the direction BuySell, "B" = Buy, "S" = Sell, uses symbol, amount, stop, limit
def enter(BuySell):
	direction = True;
	if BuySell == "S":
		direction = False;
	try:
		opentrade = con.open_trade(symbol=symbol, is_buy=direction,amount=amount, time_in_force='GTC',order_type='AtMarket',is_in_pips=True,limit=limit, stop=stop)
	except:
		print("	  Error Opening Trade.")
	else:
		print("	  Trade Opened Successfully.")

# This function closes all positions that are in the direction BuySell, "B" = Close All Buy Positions, "S" = Close All Sell Positions, uses symbol
def exit(BuySell=None):
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				print("	  Closing tradeID: " + position['tradeId'])
				try:
					closetrade = con.close_trade(trade_id=position['tradeId'], amount=position['amountK'])
				except:
					print("	  Error Closing Trade.")
				else:
					print("	  Trade Closed Successfully.")

# Returns number of Open Positions for symbol in the direction BuySell, returns total number of both Buy and Sell positions if no direction is specified
def countOpenTrades(BuySell=None):		
	openpositions = con.get_open_positions(kind='list')
	isbuy = True
	counter = 0
	if BuySell == "S":
		isbuy = False
	for position in openpositions:
		if position['currency'] == symbol:
			if BuySell is None or position['isBuy'] == isbuy:
				counter+=1
	return counter

Prepare() # Initialize strategy
StrategyHeartBeat() # Run strategy

Requesting Initial Price Data...
                     bidopen  bidclose  bidhigh   bidlow  askopen  askclose  \
date                                                                          
2020-03-24 12:30:00  1.08240   1.08169  1.08322  1.08167  1.08253   1.08184   
2020-03-24 12:35:00  1.08169   1.08309  1.08342  1.08143  1.08184   1.08323   
2020-03-24 12:40:00  1.08309   1.08265  1.08347  1.08236  1.08323   1.08278   
2020-03-24 12:45:00  1.08265   1.08215  1.08291  1.08198  1.08278   1.08230   
2020-03-24 12:50:00  1.08215   1.08176  1.08246  1.08142  1.08230   1.08191   
2020-03-24 12:55:00  1.08176   1.08216  1.08258  1.08111  1.08191   1.08231   
2020-03-24 13:00:00  1.08216   1.08252  1.08274  1.08140  1.08231   1.08266   
2020-03-24 13:05:00  1.08252   1.08334  1.08362  1.08189  1.08266   1.08349   
2020-03-24 13:10:00  1.08334   1.08422  1.08480  1.08334  1.08349   1.08436   
2020-03-24 13:15:00  1.08422   1.08484  1.08532  1.08402  1.08436   1.08498   
2020-03-24 13:20:00

Close Price: 1.08185
ColFinal: gray
ColFinal2: gray
2020-03-25 13:40:06.527370	 m5 Update Function Completed.

2020-03-25 13:45:01.518951	 m5 Bar Closed - Running Update Function...
Close Price: 1.08103
ColFinal: gray
ColFinal2: gray
2020-03-25 13:45:07.018244	 m5 Update Function Completed.

2020-03-25 13:50:01.009778	 m5 Bar Closed - Running Update Function...
Close Price: 1.08029
ColFinal: gray
ColFinal2: gray
2020-03-25 13:50:06.012370	 m5 Update Function Completed.

2020-03-25 13:55:00.996125	 m5 Bar Closed - Running Update Function...
Close Price: 1.07994
ColFinal: gray
ColFinal2: gray
2020-03-25 13:55:06.074543	 m5 Update Function Completed.

2020-03-25 14:00:01.071735	 m5 Bar Closed - Running Update Function...
Close Price: 1.07982
ColFinal: gray
ColFinal2: gray
2020-03-25 14:00:06.305739	 m5 Update Function Completed.

2020-03-25 14:05:01.314778	 m5 Bar Closed - Running Update Function...
Close Price: 1.08031
ColFinal: gray
ColFinal2: gray
2020-03-25 14:05:06.993968	 m5 Update 